# I.C.: Aprendizado por instância

Robson Mesquita Gomes  
[<robson.mesquita56@gmail>](mailto:robson.mesquita56@gmail.com)

## Introdução

Métodos de aprendizado por instância são métodos que simplesmente armazenam os exemplos de treinamento. A generalização é feita somente quando uma nova instância tiver que ser classificada.

### Referências da problemática base

Para compreender melhor a sessão a seguir é interessante que se se relembre:

 **Sistema observado:**  
 
 $$Y=G(X)$$
 
 _Onde_
 - _$G$: Processo gerador de dados, que transforma a informação $X$ em $Y$._
 
 **Aproximação:**
 
 $$\theta, \epsilon = F(X,Y)$$
 $$Ỹ = f(X, \theta) + \epsilon$$
 
 _Onde_
 - _$F$: Função de treinamento;_
 - _$\theta$: Parâmetros;_
 - _$\epsilon$: Margem de erro;_
 - _$f$: Função de aproximação._

 **Características de um Modelo I.C.**

 | Notação  |          Característica          |                            Descrição                               |
 |----------|----------------------------------|--------------------------------------------------------------------|
 |     X    | Dados de Entrada _(treinamento)_ | Dados que temos                                                    |
 |     Y    | Dados de Saída _(treinamento)_   | Dados que desejamos prever                                         |
 |     F    | Função de Treinamento            | Aprende $\theta$ a partir dos dados disponíveis                    |
 |     f    | Função de Inferência             | Usa $\theta$ para inferir $y$ a partir dos valores $x$ disponíveis |
 | $\theta$ | Parâmetros                       | Representação do "conhecimento"                                    |
 |$\epsilon$| Incerteza ($ \epsilon = Ỹ - Y $) | Margem de erro padrão das previsões realizadas                     |
 |     Ỹ    | Estimativa                       | Previsão de $Y$ (baseado em $X$) mais a incerteza $\epsilon$       |

### Lógica

**Premissa:** Entidades parecidas em alguns aspectos tendem a ser parecidas em todos os aspectos.

**SE**  
    A e B pertencem ao mesmo grupo  
**ENTÃO**    
    Alguma informação que tenho de B também se aplica a A  

### Características

- Não possui $F$, ou seja, não possui método de aprendizado
- Os modelos são os próprios dados $D$
- A função de inferência ($f$), toda vez que é utilizada, faz uma busca em $D$ e seleciona as instâncias que são mais relevantes para realizar a tarefa

## Algoritmos e aplicação

Nessa sessão discorreremos sobre alguns dos algoritmos de **Aprendizado por Instância** e seus usos práticos.

###### importações iniciais

In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import io
import requests

from collections import Counter

###### preparação para captura de dados externos

In [71]:
def get_data_by_url(url):
    
    data = requests.get(url).content
    
    return io.StringIO(data.decode('utf-8'))

### Data Frame

Para experiênciar o uso prático dos modelos e algoritmos precisaremos de uma base de dados experimental.

#### Dados Titanic

O dataset Titanic contém 5 atributos (Nome, Classe, Idade, Sexo, Sobreviveu) com 1313 instâncias, cada uma representando um dos passageiros ou tripulantes do RMS Titanic que naufragou no Oceano Atlântico em 15 de abril de 1912. 

Com esses dados queremos inferir se uma determinada pessoa sobreviveu ou não ao naufrágio ( `y = {Sobreviveu}` ), usando como base os atributos descritivos `× = {Classe, Idade, Sexo]`. Logo o problema em questão é de _Classificação_, e o atributo alvo _Sobreviveu_ contém apenas duas classes: `1`, se a pessoa sobreviveu e `0`, se a pessoa não sobreviveu.

In [72]:
# Data Frame

df = pd.read_csv(get_data_by_url('https://query.data.world/s/c2ixg24yogu6pgudqodbhn5ynrvdjr'))

df.head()

,Nome,Classe,Idade,Sexo,Sobreviveu
0,"Allen, Miss Elisabeth Walton",1.0,29.00,F,1
1,"Allison, Miss Helen Loraine",1.0,2.00,F,0
2,"Allison, Mr Hudson Joshua Creighton",1.0,30.00,M,0
3,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1.0,25.00,F,0
4,"Allison, Master Hudson Trevor",1.0,0.92,M,1


#### Melhorando a qualidade dos dados

##### Criando o atributo 'FaixaEtaria'

A fim de simplificar nossas nálises, vamos discretizar o atributo numérico `idade`, transformando-o no atribuo categórico `FaixaEtaria`, contendo as seguintes classes conforme o intervalo de idades:  

- 0 a 10: Criança
- 11 a 18: Jovem
- 19 a 50: Adulto
- +50: Idoso
- Não conhecida

In [73]:
# Transforma idade em faixa etária

def faixa_etaria(idade):
    
    if idade == np.nan:
        return 'NaoConhecida'
    
    if 0.0 <= idade <= 10.0:
        return 'Crianca'
    
    if 11.0 <= idade <= 18.0:
        return 'Jovem'
    
    if 19.0 <= idade <= 50.0:
        return 'Adulto'
    
    if idade > 50.0:
        return 'Idoso'

In [74]:
faixa_etaria(51) # <-- teste

'Idoso'

In [75]:
df['FaixaEtaria'] = [faixa_etaria(idade) for idade in df['Idade'].values]

df.head()

,Nome,Classe,Idade,Sexo,Sobreviveu,FaixaEtaria
0,"Allen, Miss Elisabeth Walton",1.0,29.00,F,1,Adulto
1,"Allison, Miss Helen Loraine",1.0,2.00,F,0,Crianca
2,"Allison, Mr Hudson Joshua Creighton",1.0,30.00,M,0,Adulto
3,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1.0,25.00,F,0,Adulto
4,"Allison, Master Hudson Trevor",1.0,0.92,M,1,Crianca


##### Seleção de atributos

Excluiremos do DataFrame os atributos `Nome` e `Idade`, pois não serão úteis em nossas nálises daqui para a frente, mantendo os atributos `Classe`, `Sexo`, `FaixaEtaria` e `Sobreviveu`.

In [76]:
df = df[['Classe', 'Sexo', 'FaixaEtaria', 'Sobreviveu']]

df.head()

,Classe,Sexo,FaixaEtaria,Sobreviveu
0,1.0,F,Adulto,1
1,1.0,F,Crianca,0
2,1.0,M,Adulto,0
3,1.0,F,Adulto,0
4,1.0,M,Crianca,1


In [77]:
df.loc[3]

Classe            1.0
Sexo                F
FaixaEtaria    Adulto
Sobreviveu          0
Name: 3, dtype: object

#### Amostrando os conjuntos de Treino e de Teste

A fim de investigar o poder de generalização dos métodos estudados, vamos separar os dados em dois conjuntos:

- **Conjunto de Treino**: Contém 70% dos dados originais e será utilizado no processo de inferência
- **Conjunto de Teste**: Contém 30% dos dados originais e será utilizado

In [78]:
total = len(df.index)

treino = df.iloc[: int(total * .7)]
teste = df.iloc[ int(total * .3) :]

In [79]:
treino.index

RangeIndex(start=0, stop=919, step=1)

### Matriz de Confusão e Acurácia

Para garantir a usabilidade dos modelos criados precisamos de métodos para calcular a precisão dos mesmos, para isso podemos utilizar a Matriz de Confusão para calcular a acurácia.

A **Matriz de Confusão** contém os valores:

- $TP$ - Verdadeiros Positivos
- $FP$ - Falsos Positivos
- $TN$ - Verdadeiros Negativos
- $FN$ - Falsos Negativos

Com esses valores definimos a métrica de **Acurácia** $A_c \in [0,1]$, tal que:  

$$ A_c = {{TP + TN} \over {TP + FP + TN + FN}} $$

In [80]:
def matriz_confusao(esperados, estimados):
    
    tp, fp, tn, fn = 0, 0, 0, 0
    
    for par in zip(esperados, estimados):
        
        if par[0] and par[1]:
            tp += 1
        
        if par[0] and not par[1]:
            fn += 1
        
        if not par[0] and not par[1]:
            tn += 1
        
        if not par[0] and par[1]:
            fp += 1

    return np.array([tp, fp, tn, fn])

In [81]:
def acuracia(esperados, estimados):
    n = len(esperados)
    
    tp, fp, tn, fn = matriz_confusao(esperados, estimados)
    
    return (tp + tn)/n

In [82]:
# teste

esperados = [0, 0, 0, 0]
estimados = [0, 1, 0, 1]

acuracia(esperados, estimados)

0.5

### kNN

Os k vizinhos mais próximos (do inglês k nearest neighbors)

**Pontos fortes**: simplicidade.  
**Pontos fracos**: custo computacional.

#### Parâmetros ($\theta$)

- $k$: número de vizinhos
- $d$: função de distância
- $m$: função de mesclagem
- Modelo = $D$

#### Simplificação

**função** kNN($k$, $D$, $x*$, $d$, $m$):  

$\forall (x,y)_i \in D$, distâncias[$i$] $\leftarrow$ d(x*,x)

distâncias $\leftarrow$ Ordenar(distâncias)

vizinhos $m$(distâncias[0:$k$])

**fimfunção**

#### Função de distância

Uma função de distância $d : \mathbb{R}^n, \mathbb{R}^n \rightarrow \mathbb{R}^+$ é uma função que calcula a similaridade de dois vetores $\alpha \in \mathbb{R}^n$ e $\beta \in \mathbb{R}^n$ através de sua distância no espaço $\mathbb{R}^n$.

No caso de instâncias em que os valores não são reais, mas categóricos, uma função de distância comum é a [Distância de Hamming](https://pt.wikipedia.org/wiki/Distância_de_Hamming), que pode ser definida como:

$$d(\alpha, \beta) = \sum_{i=0}^{n} \mathbb{I}(\alpha[i] \ne \beta[i])$$

Onde $\mathbb{I}$(expressão)$\{$retorna $1$ se expressão for verdadeira e $0$ se falsa$\}$ é a função indicadora.

##### Normalização

Na nossa implementação a Distância de Hamming é normalizado no intervalo $[0,1]$

$$d(\alpha, \beta) = {\sum_{i=0}^{n} \mathbb{I}(\alpha[i] \ne \beta[i])}\over n$$

In [83]:
def distancia_hamming(a, b, normalizar = True):
    distancia = sum( [a[key] != b[key] for key in a.keys()] );
    
    if normalizar:
        return distancia/len(a.keys())

    return distancia

In [84]:
# teste não normalizado

a = {'a':1, 'b': 0, 'c':1}
b = {'a':1, 'b': 1, 'c':1}

distancia_hamming(a,b, False)

1

In [85]:
# teste normalizado

distancia_hamming(a,b)

0.3333333333333333

##### Demonstração de aplicação

In [86]:
print("Passageiro(a) 1: \n{}\n\n{}\n\n".format( "="*16, df.iloc[1]))

print("Passageiro(a) 2: \n{}\n\n{}\n\n".format( "="*16, df.iloc[2]))

print("Distância: {}".format(distancia_hamming(df.iloc[1], df.iloc[2])))

Passageiro(a) 1: 

Classe             1.0
Sexo                 F
FaixaEtaria    Crianca
Sobreviveu           0
Name: 1, dtype: object


Passageiro(a) 2: 

Classe            1.0
Sexo                M
FaixaEtaria    Adulto
Sobreviveu          0
Name: 2, dtype: object


Distância: 0.5


#### Função de Mesclagem

In [87]:
def mais_frequente(array):
    frequencias = Counter(array)
    return max(frequencias, key=frequencias.get)

In [88]:
# teste

mais_frequente([1,1,0,0,1,0,0])

0

##### Calculando e ordenando distâncias

In [89]:
x = {'Sexo': 'M', 'FaixaEtaria': 'Adulto', 'Classe': 3.0}

###### Calculando distâncias

In [90]:
distancias = { indice: distancia_hamming(x, treino.loc[indice]) for indice in treino.index }

###### Ordenando indices por distância

In [91]:
indices = sorted(distancias, key=distancias.get)

##### Selecionando os vizinhos mais próximos e mesclando os valores

In [92]:
vizinhos = treino.loc[indices[ :3]]

vizinhos

,Classe,Sexo,FaixaEtaria,Sobreviveu
602,3.0,M,Adulto,0
607,3.0,M,Adulto,1
609,3.0,M,Adulto,1


In [93]:
y = mais_frequente(vizinhos['Sobreviveu'].values);

y

1

#### Aplicação

In [94]:
def knn(dados, k, x, y, funcao_distancia, funcao_mesclagem):
    
    distancias = { indice: funcao_distancia(x, dados.iloc[ indice ]) for indice in dados.index}
    
    indices = sorted(distancias, key=distancias.get)
    
    vizinhos = dados.loc[indices[ :k]]
    
    y_estimado = funcao_mesclagem(vizinhos[y].values)
    
    return y_estimado

In [95]:
# teste

passageiro = {'Sexo': 'F', 'FaixaEtaria': 'Adulto', 'Classe': 1.0}

knn(treino, 2, passageiro, 'Sobreviveu', distancia_hamming, mais_frequente)

1

##### Teste do modelo

In [96]:
x = ['Sexo', 'Classe', 'FaixaEtaria']
y = 'Sobreviveu'

esperados = [] # valores existentes no conjunto de testes
estimados = [] # gerados pelo kNN

for indice in teste.index:
    instancia = teste.loc[indice]
    
    esperados.append(instancia[y])
    
    consulta = {chave: instancia[chave] for chave in x}
    
    y_estimado = knn(treino, 2, consulta, y, distancia_hamming, mais_frequente)
    
    estimados.append(y_estimado)

acuracia(esperados, estimados)

0.7760869565217391

### Naïve Bayes

##### Regra de Bayes

$$P(y|x) = {{P(x|y) \cdot P(y)} \over {P(x)}}$$

_onde:_

- $x$ _são os valores dos atributos descritivos (ou a evidência)_
- $y$ _é o valor do atributo alvo (ou hipótese)_
- $P(x|y)$ _é a probabilidade à posteriore_
- $P(x|y)$ _é a verossimilhança_
- $P(y)$ _é a probabilidade à priori_
- $P(x)$ _é a probabilidade da evidência_

#### Probabilidade da evidência - $P(x)$

Probabilidade empírica:

$$P(A) = {|A| \over |\Omega|}$$

In [97]:
# Tamanho total do conjunto de dados
n = len(treino.index)

# Atributos descritivos
x = "Sexo == 'M' & Classe == 3"

# Filtra o conjunto de dados a partir dos atributos descritivos
x_dados = treino.query(x)

# Tamanho total do filtro
xn = len(x_dados.index)

# Probabilidade Empírica P(x)
px = xn/n

px

0.24156692056583243

#### Probabilidade à priori - $P(y)$

Para o caso do algoritmo de Naïve Bayes utilizaremos um dicionário para armazenar todas as probabilidades conhecidas obtendo-as a partir de Probabilidade Empírica.

In [98]:
# Número de classes do atributo alvo
classes = treino['Sobreviveu'].unique()

classes

array([1, 0])

In [99]:
# Dicionário que conterá as probabilidades P(y) para cada classe de y
py = {}

for classe in classes:
    
    # Filtra o conjunto de dados a partir dos atributos alvo
    y_dados = treino.query("Sobreviveu == {}".format(classe))
    
    # Tamanho do filtro
    yn = len(y_dados.index)
    
    # Probabilidade à priori da classe y (P(y))
    py[classe] = yn/n
    

py

{1: 0.42219804134929273, 0: 0.5778019586507073}

#### Verossimilhança - $P(x|y)$

$$P(x|y) = {P(x,y) \over P(y)}$$

In [100]:
# Dicionário que conterá as probabilidades P(x|y) para cada classe de y
pxy = {}

for classe in classes:
    
    # Filtra o conjunto de dados a partir dos atributos descritivos e dos atributos alvo
    xy_dados = treino.query("{} & Sobreviveu == {}".format(x, classe))
    
    # Tamanho do filtro
    xyn = len(xy_dados.index)
    
    # Verossimilhança dos atributos descritivos em relação aos atributos alvo
    pxy[classe] = (xyn/n)/py[classe]

pxy

{1: 0.07731958762886598, 0: 0.3615819209039548}

#### Probabilidade à Posteriori - $P(y|x)$

$$P(y|x) = {{P(x|y) \cdot P(y)} \over {P(x)}}$$

In [101]:
# Dicionário que conterá as probabilidades P(y|x) para cada classe de y
pyx = {}

for classe in classes:
    # Regra de Bayes P(y|x) = (P(x|y)*P(y))/P(x)
    pyx[classe] = (pxy[classe]*py[classe])/px

pyx

{1: 0.13513513513513514, 0: 0.8648648648648649}

#### Máximo à Posteriori - $MAP$

A classificação é decidida pela classe $y_i$ que maximizar a probabilidade à posteriori $P(y_i|x)$  

$$y_{MAP} = \arg\max_{y_i}P(y_i|x) $$

In [102]:
max(pyx, key=pyx.get)

0

#### Algorítmo

In [103]:
def naiveBayes(dados, x, y):
    n = dados.size
    
    # Probabilidade da evidência P(x)
    x_dados = dados.query(x)
    xn = x_dados.size
    px = xn/n
    
    # Probabilidade à priori P(y) por classe
    classes = dados[y].unique()
    py = {}
    
    for classe in classes:
    
        y_dados = dados.query("{} == {}".format(y, classe))
        yn = y_dados.size
        
        py[classe] = yn/n
    
    # Verossimilhança P(x|y) por classe
    pxy = {}
    
    for classe in classes:
        xy_dados = dados.query("{} & {} == {}".format(x, y, classe))
        xyn = xy_dados.size
        
        pxy[classe] = (xyn/n)/py[classe]
    
    # Probabilidade à posteriori - P(y|x) por classe
    pyx = {}
    
    for classe in classes:
        # Teorema de Bayes P(y|x) = (P(x|y)*P(y))/P(x)
        pyx[classe] = (pxy[classe]*py[classe])/px
    
    return max(pyx, key=pyx.get)

In [104]:
# teste

naiveBayes(treino, "Sexo == 'F' & Classe == 1", 'Sobreviveu')

1

In [105]:
naiveBayes(treino, "FaixaEtaria == 'Adulto' & Classe == 3", 'Sobreviveu')

0

##### Teste do modelo

In [106]:
x = ['Sexo', 'Classe', 'FaixaEtaria']
y = 'Sobreviveu'

esperados = []
estimados = []

for indice in teste.index:
    instancia = teste.loc[indice]
    
    esperados.append(instancia[y])
    
    tmp = [ "{} == '{}'".format(chave, instancia[chave]) for chave in x ]
    consulta = ' & '.join(tmp)
    
    y_estimado = naiveBayes(teste, consulta, y)
    
    estimados.append(y_estimado)

acuracia(esperados, estimados)

/var/folders/2y/q957rqmj3c1ckz7t4qg_1cdr0000gn/T/ipykernel_76154/3069504419.py:34: RuntimeWarning: invalid value encountered in scalar divide
  pyx[classe] = (pxy[classe]*py[classe])/px


0.24565217391304348

[<< Tópico Anterior](00-introducao.ipynb)